In [1]:
from typing import Tuple
import pandas as pd
import numpy as np


In [3]:
class Admensional:
    
    @staticmethod
    def reynolds(diametro:float, rotacao:float, densidade:float, viscosidade_dinamica:float) -> float:
        return rotacao*diametro**2*densidade/viscosidade_dinamica
    
    @staticmethod
    def pow_number(potencia:float, rotacao:float, diametro:float, densidade:float) -> float:
        return potencia/(rotacao**3*diametro**5*densidade)

In [4]:
class Agitação:
    
    @staticmethod
    def __calcula_potencia_com_numero_de_potencia(pow_number:float, rotacao:float, diametro:float, densidade:float) -> float:
        return pow_number*rotacao**3*diametro**5*densidade
    
    @staticmethod
    def __calcula_pow_number_by_reynolds(reynolds:float, impeler: str = 'pow_number_pitched_blade.xlsx') -> float:
        df = df = pd.read_excel('dados/' + impeler)
        max_reynold, max_pot = df[df['Reynolds']>=reynolds].values[0]
        mim_reynold, mim_pot = df[df['Reynolds']<reynolds].values[-1]

        N = max_pot - (max_reynold-reynolds)*(max_pot-mim_pot)/(max_reynold-mim_reynold)

        return N  
    @staticmethod
    def calculate_potencia(diametro:float, rotacao:float, densidade:float, viscosidade_dinamica:float, 
                           impeler: str = 'pow_number_pitched_blade.xlsx')-> float:
        reynolds = Admensional.reynolds(diametro=diametro, rotacao=rotacao, 
                                        densidade=densidade, viscosidade_dinamica=viscosidade_dinamica)
        numero_de_potencia = Agitação.__calcula_pow_number_by_reynolds(reynolds=reynolds, impeler=impeler)
        
        potencia = Agitação.__calcula_potencia_com_numero_de_potencia(pow_number=numero_de_potencia,
                                                                     rotacao=rotacao, diametro=diametro,
                                                                     densidade=densidade)
        
        return potencia
    
    @staticmethod
    def aumento_escala_por_semelhança_geometrica(potencia:float, rotacao1:float, rotacao2:float, diametro1:float,diametro2:float):
        return potencia*rotacao1**3*diametro1**5/(rotacao2**3*diametro2**5)
    
    @staticmethod
    def projeta_novo_tanque(diametro:float, rotacao: float, densidade:float, 
                            viscosidade_dinamica:float, impeler:str = 'pow_number_pitched_blade.xlsx') -> Tuple[float, dict]:
        dimensoes = {'diametro': round(diametro,4),
                     'altura': round(diametro,4),
                     'impelidor': round(diametro/3,4),
                     'altura_impelidor': round(diametro/3,4),
                     'chicana': round(diametro/10,4)}
        mixing_energy = Agitação.calculate_potencia(diametro=.1, rotacao=rotacao, densidade=densidade,
                                                    viscosidade_dinamica=viscosidade_dinamica)
        mixing_energy = Agitação.aumento_escala_por_semelhança_geometrica(mixing_energy, rotacao, rotacao, diametro/3, .1)
        return mixing_energy, dimensoes

In [5]:
rotacao = 4.1 # -> s-1
densidade = 1000 # -> kg/m³
viscosidade_dinamica = 0.001 # -> Pas*s

volume = 2
diametro = (4*volume/(np.pi))**(1/3)

mixing_energy, dimensoes = Agitação.projeta_novo_tanque(diametro, rotacao, densidade, viscosidade_dinamica)

In [8]:
print('ENERGIA DE AGITAÇÃO[W]: %s  E TANQUE DE DIMENSÕES[m] %s ' %(mixing_energy, dimensoes))

ENERGIA DE AGITAÇÃO[W]: 2020.2420795938351  E TANQUE DE DIMENSÕES[m] {'diametro': 1.3656, 'altura': 1.3656, 'impelidor': 0.4552, 'altura_impelidor': 0.4552, 'chicana': 0.1366} 
